# L3: Multi-agent Customer Support Automation

In this lesson, you will learn about the six key elements which help make Agents perform even better:
- Role Playing
- Focus
- Tools
- Cooperation
- Guardrails
- Memory

In [33]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, API and LLM

In [56]:
from crewai import Agent, Task, Crew

In [57]:
from dotenv import load_dotenv
load_dotenv()

True

In [58]:
from crewai import LLM

llm = LLM(
    model="gemini/gemini-2.0-flash",
    temperature=0.0
)

## Role Playing, Focus and Cooperation

In [59]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at Health Benefits Desk and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True,
    llm=llm
)

- By not setting `allow_delegation=False`, `allow_delegation` takes its default value of being `True`.
- This means the agent _can_ delegate its work to another agent which is better suited to do a particular task. 

In [60]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at Health Benefits Desk  and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True,
    llm=llm
)

* **Role Playing**: Both agents have been given a role, goal and backstory.
* **Focus**: Both agents have been prompted to get into the character of the roles they are playing.
* **Cooperation**: Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together.

## Tools, Guardrails and Memory

### Tools

- Import CrewAI tools

In [61]:
from crewai_tools import PDFSearchTool

### Possible Custom Tools
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
- ... and more

- Some ways of using CrewAI tools.

```Python
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
```

- Instantiate a document scraper tool.
- The tool will search from the provided pdf file

In [62]:
import os

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

doc_srch_tool = PDFSearchTool(
    pdf='pdf/Insurance_Plan.pdf',
    config=dict(
        llm=dict(
            provider="google",
            config=dict(
                model="gemini/gemini-2.0-flash",
                api_key=GEMINI_API_KEY,
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="google",
            config=dict(
                model="models/embedding-001",
                task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)

##### Different Ways to Give Agents Tools

- Agent Level: The Agent can use the Tool(s) on any Task it performs.
- Task Level: The Agent will only use the Tool(s) when performing that specific Task.

**Note**: Task Tools override the Agent Tools.

### Creating Tasks
- You are passing the Tool on the Task Level.

In [63]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[doc_srch_tool],
    agent=support_agent,
)

- `quality_assurance_review` is not using any Tool(s)
- Here the QA Agent will only review the work of the Support Agent

In [64]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


### Creating the Crew

#### Memory
- Setting `memory=True` when putting the crew together enables Memory.

In [65]:
crew = Crew(
    agents=[support_agent, support_quality_assurance_agent],
    tasks=[inquiry_resolution, quality_assurance_review],
    verbose=True
)

### Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

#### Guardrails
- By running the execution below, you can see that the agents and the responses are within the scope of what we expect from them.

In [66]:
inputs = {
    "customer": "USTech",
    "person": "Praveen Nair",
    "inquiry": "I need help with understanding my dedeuctible for the current year. "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 1f0476d3-0927-44f1-b634-527da7d23c55                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Support Representative                                                                           │
│                                                                                                                 │
│  Task: USTech just reached out with a super important ask:                                                      │
│  I need help with understanding my dedeuctible for the current year. Can you provide guidance?                  │
│                                                                                                                 │
│  Praveen Nair from USTech is the one that reached out. Make sure to use everything you know to provide the      │
│  best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Support Representative                                                                           │
│                                                                                                                 │
│  Thought: Okay, I understand. Praveen Nair from USTech needs help understanding their deductible for the        │
│  current year. I need to use the available tool to search the insurance plan document and provide a detailed    │
│  and friendly explanation.                                                                                      │
│                                                                                                                 │
│  Using Tool: Search a PDF's content                                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"query\": \"deductible\"}"                                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Relevant Content:                                                                                              │
│  amount.                                                                                                        │
│                                                                                                                 │
│  the cost; the plan pays the rest.                                                                              │
│                                                                                                                 │
│  deductible. Then, you pay 20% of the allowed amount; the plan pays the remainder of the allowed amount.        │
│  Regardless of whether you receive services in-network or out-of-network, this benefit is subject to your in-   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: aa3338f6-d523-411f-9008-3aa571941b87                                                                     │
│  Agent: Senior Support Representative                                                                           │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Support Quality Assurance Specialist                                                                    │
│                                                                                                                 │
│  Task: Review the response drafted by the Senior Support Representative for USTech's inquiry. Ensure that the   │
│  answer is comprehensive, accurate, and adheres to the high-quality standards expected for customer support.    │
│  Verify that all parts of the customer's inquiry have been addressed thoroughly, with a helpful and friendly    │
│  tone.                                                                                                          │
│  Check for references and sources used to  find the information, ensuring the response is well-supported and    │
│  leaves no questions unanswered.                                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Support Quality Assurance Specialist                                                                    │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Hey USTech Team,                                                                                               │
│                                                                                                                 │
│  Thanks for reaching out with your question about network deductibles! I'm happy to clarify how this works.     │
│                                                                                                                 │
│  Essentially, a deductible is the amount you pay out-of-pocket for covered healthcare services before your      │
│  health plan starts to pay. For services within your network, once you've met your in-network deductible, your  │
│  plan starts sharing the cost with you.                                                                         │
│                                                                                                                 │
│  Based on the information we have, here’s a breakdown of the different ways your plan might pay after you meet  │
│  your in-network deductible. Note that these are all possibilities, and your specific plan will determine the   │
│  exact split:                                                                                                   │
│                                                                                                                 │
│  *   **The plan pays 100%:** After you meet your deductible, the plan covers all the costs for in-network       │
│  services. You pay nothing!                                                                                     │
│                                                                                                                 │
│  *   **The plan pays 90%, you pay 10%:** After your deductible, the plan covers 90% of the allowed amount, and  │
│  you are responsible for the remaining 10%.                                                                     │
│                                                                                                                 │
│  *   **The plan pays 85%, you pay 15%:** You pay 15% of the allowed amount after the deductible is met.         │
│                                                                                                                 │
│  *   **The plan pays 80%, you pay 20%:** This is a common scenario where you pay 20% of the allowed amount for  │
│  services after meeting your deductible.                                                                        │
│                                                                                                                 │
│  *   **The plan pays 75%, you pay 25%:** The plan covers three-quarters of the cost once you've met your        │
│  deductible.                                                                                                    │
│                                                                                                                 │
│  *   **The plan pays 70%, you pay 30%:** You are responsible for 30% of the costs after your deductible is      │
│  met.                                                                                                           │
│                                                                                                                 │
│  *   **The plan pays 65%, you pay 35%:** The plan cover

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 50c96299-fb9e-402b-a2f5-d85ce32cf55e                                                                     │
│  Agent: Support Quality Assurance Specialist                                                                    │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 1f0476d3-0927-44f1-b634-527da7d23c55                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Hey USTech Team,                                                                                 │
│                                                                                                                 │
│  Thanks for reaching out with your question about network deductibles! I'm happy to clarify how this works.     │
│                                                                                                                 │
│  Essentially, a deductible is the amount you pay out-of-pocket for covered healthcare services before your      │
│  health plan starts to pay. For services within your network, once you've met your in-network deductible, your  │
│  plan starts sharing the cost with you.                                                                         │
│                                                                                                                 │
│  Based on the information we have, here’s a breakdown of the different ways your plan might pay after you meet  │
│  your in-network deductible. Note that these are all possibilities, and your specific plan will determine the   │
│  exact split:                                                                                                   │
│                                                                                                                 │
│  *   **The plan pays 100%:** After you meet your deductible, the plan covers all the costs for in-network       │
│  services. You pay nothing!                                                                                     │
│                                                                                                                 │
│  *   **The plan pays 90%, you pay 10%:** After your deductible, the plan covers 90% of the allowed amount, and  │
│  you are responsible for the remaining 10%.                                                                     │
│                                                                                                                 │
│  *   **The plan pays 85%, you pay 15%:** You pay 15% of the allowed amount after the deductible is met.         │
│                                                                                                                 │
│  *   **The plan pays 80%, you pay 20%:** This is a common scenario where you pay 20% of the allowed amount for  │
│  services after meeting your deductible.                                                                        │
│                                                                                                                 │
│  *   **The plan pays 75%, you pay 25%:** The plan covers three-quarters of the cost once you've met your        │
│  deductible.                                                                                                    │
│                                                                                                                 │
│  *   **The plan pays 70%, you pay 30%:** You are responsible for 30% of the costs after your deductible is      │
│  met.                                                                                                           │
│                                                       

- Display the final result as Markdown.

In [68]:
from IPython.display import Markdown
Markdown(result.raw)

Hey USTech Team,

Thanks for reaching out with your question about network deductibles! I'm happy to clarify how this works.

Essentially, a deductible is the amount you pay out-of-pocket for covered healthcare services before your health plan starts to pay. For services within your network, once you've met your in-network deductible, your plan starts sharing the cost with you.

Based on the information we have, here’s a breakdown of the different ways your plan might pay after you meet your in-network deductible. Note that these are all possibilities, and your specific plan will determine the exact split:

*   **The plan pays 100%:** After you meet your deductible, the plan covers all the costs for in-network services. You pay nothing!

*   **The plan pays 90%, you pay 10%:** After your deductible, the plan covers 90% of the allowed amount, and you are responsible for the remaining 10%.

*   **The plan pays 85%, you pay 15%:** You pay 15% of the allowed amount after the deductible is met.

*   **The plan pays 80%, you pay 20%:** This is a common scenario where you pay 20% of the allowed amount for services after meeting your deductible.

*   **The plan pays 75%, you pay 25%:** The plan covers three-quarters of the cost once you've met your deductible.

*   **The plan pays 70%, you pay 30%:** You are responsible for 30% of the costs after your deductible is met.

*   **The plan pays 65%, you pay 35%:** The plan covers 65% of the costs after you meet your deductible.

*   **The plan pays 60%, you pay 40%:** You will pay 40% of the allowed amount for services once you've met your deductible.

*   **The plan pays 55%, you pay 45%:** After meeting your deductible, you'll be responsible for 45% of the costs.

*   **The plan pays 50%, you pay 50%:** You and the plan share the costs equally after you've met your deductible.

*   **The plan pays 45%, you pay 55%:** You are responsible for 55% of the costs after your deductible is met.

*   **The plan pays 40%, you pay 60%:** The plan covers 40% of the costs after you meet your deductible.

*   **The plan pays 35%, you pay 65%:** You will pay 65% of the allowed amount for services once you've met your deductible.

*   **The plan pays 30%, you pay 70%:** You are responsible for 70% of the costs after your deductible is met.

*   **The plan pays 25%, you pay 75%:** You will pay 75% of the allowed amount for services once you've met your deductible.

**Important Considerations:**

*   **Allowed Amount:** The percentages are based on the "allowed amount," which is the maximum amount your plan will pay for a covered health care service. This may be different from the provider's actual charge.
*   **In-Network vs. Out-of-Network:** Remember that these cost-sharing arrangements apply to in-network services. Out-of-network services usually have a different deductible and/or coinsurance structure, which may result in you paying a higher portion of the cost.
*   **Specific Plan Details:** To know the *exact* cost-sharing arrangement for your specific plan, please check your plan documents (like your Summary of Benefits) or log in to your account on our website. You can also call the number on your insurance card, and a representative can walk you through your benefits.

I hope this helps clarify how your network deductible works! Let me know if you have any other questions. We're always here to help!